In [1]:
import pandas  as pd
import networkx as nx
import re
import json
import matplotlib.pyplot as plt

## DATA THINGY

In [2]:
#Our flitered dataset

In [3]:
data = pd.read_csv("../../Data/final_cool_dataset.csv")

In [4]:
#Coordinates from 1st dataset

In [5]:
coordinates_df = pd.read_csv("../../Data/Airports.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
coordinates_df = coordinates_df.rename(columns={'Y': 'LAT', 'X': 'LONG'})
coordinates_df = coordinates_df[["LONG","LAT","Loc_Id"]]

In [7]:
s_1 = set(coordinates_df["Loc_Id"])

In [8]:
#2nd dataset

In [9]:
new_names = pd.read_csv("../../Data/us-airports.csv")
new_names = new_names[["latitude_deg","longitude_deg","local_code"]]
new_names = new_names.iloc[1: , :]
new_names = new_names[new_names['local_code'].notna()]
new_names = new_names.rename(columns = {"local_code" :"ORIGIN"})

In [10]:
s_2 = set(new_names["ORIGIN"])

## Airports which we do not have coordinates for

In [11]:
all_airpor_keys = s_1.union(s_2)
set(data["ORIGIN"]) - all_airpor_keys

{'ABL',
 'AET',
 'AIN',
 'AKB',
 'AOS',
 'ATT',
 'BFB',
 'BFT',
 'BKC',
 'BKG',
 'BLD',
 'CDL',
 'CIL',
 'CKD',
 'CLD',
 'CLU',
 'CYF',
 'CYM',
 'DGB',
 'DIO',
 'DJN',
 'DOF',
 'DRF',
 'DRG',
 'EDA',
 'EGX',
 'EMK',
 'ESD',
 'FCA',
 'FID',
 'FRD',
 'FRY',
 'FWL',
 'GEK',
 'GMT',
 'HAR',
 'HBH',
 'HGZ',
 'HHH',
 'HKB',
 'HLI',
 'HSH',
 'HSL',
 'IRC',
 'ISS',
 'KBC',
 'KCG',
 'KCQ',
 'KEH',
 'KGK',
 'KKH',
 'KLL',
 'KLN',
 'KLW',
 'KMO',
 'KNK',
 'KOT',
 'KOZ',
 'KPM',
 'KPN',
 'KPV',
 'KUK',
 'KWN',
 'LIJ',
 'LIV',
 'LKE',
 'LMA',
 'LNI',
 'LPS',
 'LVD',
 'MDR',
 'MEO',
 'MLL',
 'MNT',
 'MQT',
 'MVW',
 'MXY',
 'NAD',
 'NCN',
 'NHZ',
 'NIB',
 'NKI',
 'NLG',
 'NME',
 'NNK',
 'NNL',
 'NPT',
 'NUP',
 'ODW',
 'OHC',
 'OLH',
 'ORV',
 'OTS',
 'PBK',
 'PCT',
 'PDB',
 'PGC',
 'PIP',
 'PML',
 'PPV',
 'PQS',
 'PTA',
 'PUO',
 'RBH',
 'RBN',
 'RDB',
 'RLU',
 'RSJ',
 'SCE',
 'SCF',
 'SKK',
 'STG',
 'SYB',
 'TEH',
 'TEK',
 'TLA',
 'TNK',
 'TWA',
 'TWD',
 'UGI',
 'UST',
 'UTM',
 'UUK',
 'VIK',
 'WAA',


In [12]:
#Merge coordinates to dataset

In [13]:
data = data.merge(new_names, on = "ORIGIN")
data = data.rename(columns = {"latitude_deg": "ORIGIN_LAT", "longitude_deg": "ORIGIN_LONG"})
new_names = new_names.rename(columns = {"ORIGIN" :"DEST"})
data = data.merge(new_names, on = "DEST")
data = data.rename(columns = {"latitude_deg": "DEST_LAT", "longitude_deg": "DEST_LONG"})

In [14]:
data

,ORIGIN,DEST,PAYLOAD,DISTANCE,SEATS,PASSENGERS,DEPARTURES_PERFORMED,CARRIER_NAME,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,ORIGIN_NAME,DEST_NAME,ORIGIN_LAT,ORIGIN_LONG,DEST_LAT,DEST_LONG
0,ANC,OME,33729908.0,539.0,31762.0,20623.0,1133.0,['Era Aviation' 'Alaska Airlines Inc.'\n 'Tato...,10299.0,13873.0,"Anchorage, AK: Ted Stevens Anchorage Internati...","Nome, AK: Nome Airport",61.1744,-149.996002,64.51219940185547,-165.44500732421875
1,ATK,OME,4500.0,463.0,19.0,0.0,1.0,['Hageland Aviation Service'],10396.0,13873.0,"Atqasuk, AK: Atqasuk Edward Burnell Sr. Memorial","Nome, AK: Nome Airport",70.46704,-157.436013,64.51219940185547,-165.44500732421875
2,AUK,OME,39300.0,129.0,26.0,7.0,9.0,['Arctic Transportation' 'Bering Air Inc.' 'Ha...,10416.0,13873.0,"Alakanuk, AK: Alakanuk Airport","Nome, AK: Nome Airport",62.680042266799994,-164.659927368,64.51219940185547,-165.44500732421875
3,BET,OME,84200.0,283.0,346.0,260.0,18.0,['Hageland Aviation Service' 'Bering Air Inc.'...,10551.0,13873.0,"Bethel, AK: Bethel Airport","Nome, AK: Nome Airport",60.77980042,-161.8379974,64.51219940185547,-165.44500732421875
4,BRW,OME,59180.0,520.0,253.0,132.0,13.0,['Bering Air Inc.' 'Era Aviation' 'Hageland Av...,10754.0,13873.0,"Barrow, AK: Wiley Post/Will Rogers Memorial","Nome, AK: Nome Airport",71.285402,-156.766008,64.51219940185547,-165.44500732421875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19004,MTM,KPB,1850.0,116.0,6.0,0.0,1.0,['Venture Travel LLC d/b/a Taquan Air Service'],13504.0,12773.0,"Metlakatla, AK: Metlakatala","Point Baker, AK: Point Baker Airport",55.131001,-131.578003,56.3518981934,-133.623001099
19005,LGD,HES,6900.0,71.0,0.0,0.0,2.0,['Empire Airlines Inc.' 'Federal Express Corpo...,12956.0,12103.0,"La Grande, OR: La Grande/Union County","Hermiston, OR: Hermiston Municipal",45.2901992798,-118.007003784,38.6534996033,-122.899002075
19006,OXR,CCR,2000.0,306.0,8.0,3.0,1.0,['Avjet Corporation'],13988.0,10910.0,"Oxnard/Ventura, CA: Oxnard","Concord, CA: Buchanan Field",34.200801849365,-119.20700073242,37.9897003174,-122.056999207
19007,HAO,UOX,2000.0,442.0,8.0,2.0,1.0,['Avjet Corporation'],12068.0,15481.0,"Hamilton, OH: Butler County Regional Hogan Field","Oxford, MS: University-Oxford",39.363800048799995,-84.5220031738,34.384300231934,-89.536796569824


## Create json lookup tables

In [15]:
#Abbreviation (Local code) 

In [16]:
abv_list = list(zip(data["ORIGIN"],data["ORIGIN_LAT"], data["ORIGIN_LONG"]))

In [17]:
abv_lookup_table = dict()
for elm in abv_list:
    abv_lookup_table[elm[0]] = (elm[1], elm[2])

In [18]:
with open("../../Data/abv_lookup_table.json", "w") as outfile:
    json.dump(abv_lookup_table, outfile)

In [19]:
#Names (Full airport name incl state)

In [20]:
name_list = list(zip(data["ORIGIN_NAME"],data["ORIGIN_LAT"], data["ORIGIN_LONG"]))

In [21]:
airport_lookup_table = dict()
for elm in name_list:
    airport_lookup_table[elm[0]] = (elm[1], elm[2])

In [22]:
with open("../../Data/airport_names_lookup_table.json", "w") as outfile:
    json.dump(airport_lookup_table, outfile)

## Graph

In [23]:
edges = [(row['ORIGIN_NAME'], row['DEST_NAME'], {'weight': row['PASSENGERS']}) for _, row in data.iterrows()]

In [24]:
G = nx.DiGraph()
G.add_edges_from(edges)

In [25]:
nx.is_strongly_connected(G)

False

In [26]:
#Isolated nodes

In [27]:
components = sorted(nx.strongly_connected_components(G), key=len, reverse=True)
components[1:]

[{'Circle Hot Springs, AK: Circle Hot Springs Airport'}]

In [28]:
to_removed = [list(x) for x in components  if x != max(components)]
flat_list = [item for sublist in to_removed for item in sublist]
G.remove_nodes_from(flat_list)
nx.is_strongly_connected(G)

True

Here we find that the diameter of the graph is 7
This means that you can get from any airport to any airport in at most 7 steps. 
The longest shortest path is 7

In [29]:
nx.diameter(G)

7

This is the average shortest path of the network. Meaning on average if you pick two nodes at random the distance is a path with the length of 3

In [30]:
nx.average_shortest_path_length(G)

2.975030774994648

Finding the longest shortest paths

In [31]:
li = []
for node in G.nodes():
    start_end_nodes = [(node, k) for k,v in nx.shortest_path_length(G, node).items() if v == 7]
    if start_end_nodes != []:
        li.append(start_end_nodes)

In [32]:
routes = dict()
coord = dict()
for elm in li:
    from_ = elm[0][0]
    #routes[from_] = []
    routes_dict = dict()
    coord_dict = dict()
    for i in elm:
        to_ = i[1]
        shortest_path = nx.shortest_path(G, from_, to_)
        
        routes_dict[to_] = shortest_path
        coord_dict[to_] = [airport_lookup_table[x] for x in shortest_path]
        #airport_name_coordinates
        #coord_dict[to_] = [airport_name_coordinates(x) for x in shortest_path]
    coord[from_] = coord_dict
    routes[from_] = routes_dict

In [34]:
from_ = []
dest_ = []
coordinates = []
path = []
for elm in coord.items():
    fr = elm[0]
    
    for i in elm[1].keys():
        dest_.append(i)
        path.append(routes[fr][i])
        from_.append(fr)
    for i in elm[1].values():
        coordinates.append(i)

In [35]:
d = {"from": from_, "dest": dest_, "path": path, "coordinates": coordinates}
df_paths = pd.DataFrame(data=d)
#df_paths.to_csv('../../Data/to_florian_dataframe_longest_paths.csv', index=False)

In [54]:
float_coords = []
for i in df_paths['coordinates']:
    temp = []
    for j in i:
        j_0 = [eval(j[0])]
        j_1 = [eval(j[1])]
        temp.append(list(zip(j_0,j_1)))
    float_coords.append(temp)

In [55]:
float_coords

[[[(62.320499420166, -150.09399414062)],
  [(61.7881011963, -157.350006104)],
  [(60.77980042, -161.8379974)],
  [(61.1744, -149.996002)],
  [(33.6367, -84.428101)],
  [(40.27669906616211, -74.8134994506836)],
  [(41.47859954834, -73.135200500488)],
  [(38.942902, -76.568398)]],
 [[(62.320499420166, -150.09399414062)],
  [(61.7881011963, -157.350006104)],
  [(60.77980042, -161.8379974)],
  [(64.81510162, -147.8560028)],
  [(41.9786, -87.9048)],
  [(42.74829864501953, -73.80169677734375)],
  [(41.66930008, -70.28040314)],
  [(44.53459930419999, -72.6139984131)]],
 [[(62.320499420166, -150.09399414062)],
  [(61.7881011963, -157.350006104)],
  [(60.77980042, -161.8379974)],
  [(64.81510162, -147.8560028)],
  [(41.9786, -87.9048)],
  [(41.06700134277344, -73.70760345458984)],
  [(39.168300628699996, -77.1660003662)],
  [(39.92750168, -74.29239655)]],
 [[(62.320499420166, -150.09399414062)],
  [(61.7881011963, -157.350006104)],
  [(60.77980042, -161.8379974)],
  [(64.81510162, -147.8560028)

In [53]:
for i in df_paths['coordinates']:
    print(i)
    break

[('62.320499420166', '-150.09399414062'), ('61.7881011963', '-157.350006104'), ('60.77980042', '-161.8379974'), ('61.1744', '-149.996002'), ('33.6367', '-84.428101'), ('40.27669906616211', '-74.8134994506836'), ('41.47859954834', '-73.135200500488'), ('38.942902', '-76.568398')]


Centrality

In graph theory and network analysis, indicators of centrality assign numbers or rankings to nodes within a graph corresponding to their network position. Applications include identifying the most influential person(s) in a social network,

In [ ]:
sorted_degree_centrality = sorted(nx.degree_centrality(G).items(), key=lambda x: x[1], reverse=True)
sorted_degree_centrality = [x[0] for x in sorted_degree_centrality[0:20]]
#These are the most "central" nodes in the graph
sorted_degree_centrality

In [ ]:
mask = data["ORIGIN_NAME"].isin(sorted_degree_centrality)
to_florian_dataframe_degree_centrality = data[mask][["ORIGIN_NAME","ORIGIN_LAT", "ORIGIN_LONG"]].drop_duplicates('ORIGIN_NAME', keep='last')
to_florian_dataframe_degree_centrality.to_csv('../../Data/to_florian_dataframe_degree_centrality.csv', index=False)

Clustering coefficient

In [ ]:
#The average clustering coefficient of the network is 0.56
nx.average_clustering(G)

In [ ]:
#Clustering for nodes
clustering = nx.clustering(G)
clustering_sorted = sorted(clustering.items(), key=lambda x: x[1], reverse=True)
clustering_sorted

In [ ]:
data

In [ ]:
li = []
coef = []
for elm in clustering_sorted:
    if elm[1] in [1,0]:
        li.append(elm[0])
        coef.append(elm[1])

In [ ]:
d = {"ORIGIN_NAME": li, "COEFFICIENT": coef}
df_coef = pd.DataFrame(data=d)

In [ ]:
mask = data["ORIGIN_NAME"].isin(li)

In [ ]:
to_florian_dataframe = data[mask][["ORIGIN_NAME","ORIGIN_LAT","ORIGIN_LONG"]].drop_duplicates('ORIGIN_NAME', keep='last')

In [ ]:
to_florian_dataframe = pd.merge(to_florian_dataframe, df_coef, on = ["ORIGIN_NAME"])

In [ ]:
to_florian_dataframe.to_csv('../../Data/to_florian_clustering_coefficient.csv', index=False)

In [ ]:
#Clustering coefficient plottet

In [ ]:
t = [x[1] for x in clustering_sorted]
plt.hist(t, bins = 50);

In [ ]:
#Degree distribution

In [ ]:

def plot_degree_dist(G):
    degrees = [G.degree(n) for n in G.nodes()]
    plt.hist(degrees, bins = 30)
    plt.show()

plot_degree_dist(G)

In [ ]:
#Degree distribution
def plot_degree_dist(G):
    degrees = [G.degree(n) for n in G.nodes()]
    plt.loglog(degrees)
    plt.show()

plot_degree_dist(G)

In [ ]:
#Degree distribution
#It does not follow the power law
def plot_degree_dist(G):
    degrees = [n[1] for n in G.degree]
    plt.loglog(degrees)
    plt.gca().invert_yaxis()
    plt.show()

plot_degree_dist(G)